In [68]:
import os
import sys
import glob
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import importlib

# Forçar reload completo do módulo utils
if 'utils' in sys.modules:
    del sys.modules['utils']
from utils import *

file_path='/Users/jtakeo/googleDrive/myProjects/finalPartSpec/surigae/sarspec/sar-data/dataset-wav-sar-l3-spc-nrt-global-s1a_20210427T0600Z_20210526T1800Z_P20210517T0541Z_27-APR-2021-06H_lon_+158_lat_+41-rep.nc'
grp = "obs_params"
index=223
date_time="2021-04-28T20:00:00Z"  # WW3 reference time
ds = xr.open_dataset(file_path, group=grp)

# Carregar espectro SAR e converter para m²·s·rad⁻¹
E2d, freq, dirs, dirs_rad, selected_time = load_sar_spectrum(ds, date_time, index)

# Calcular parâmetros
hs, tp, dp, m0, delf, ddir, _, _ = calculate_wave_parameters(E2d, freq, dirs_rad)

print('\n' + '='*70)
print(' RESUMO FINAL - COMPARAÇÃO SAR vs WW3')
print('='*70)
print(f"Timestamp selecionado: {selected_time}")
print(f"Quality flag: {ds.L2_partition_quality_flag[abs(ds.time.values - np.datetime64(selected_time)).argmin()].values}")
print('-'*70)
print(f"SAR calculado:")
print(f"  Hs = {hs:.2f} m, Tp = {tp:.2f} s, Dp = {dp:.0f}°")
print(f"\nSAR do arquivo (VAVH/VTPK/VPED):")
idx_sar = abs(ds.time.values - np.datetime64(selected_time)).argmin()
print(f"  Hs = {ds.VAVH[idx_sar].values:.2f} m, Tp = {ds.VTPK[idx_sar].values:.2f} s, Dp = {ds.VPED[idx_sar].values:.0f}°")
print(f"\nWW3 (referência para este horário):")
print(f"  Hs = 2.74 m, Tp = 12.8 s, Dp = 195°")
print('='*70)

Variáveis disponíveis no arquivo SAR: ['VAVH', 'VPED', 'VTPK', 'time', 'longitude', 'latitude', 'wave_spec', 'partition_domain_spec', 'wavenumber_spec', 'direction_spec', 'nv', 'inten', 'snr', 'resolution_spec', 'VPED_flag', 'VTPK_flag', 'VAVH_flag', 'L2_source_product', 'L2_partition_quality_flag']
Usando arquivo preprocessado (CMEMS), shape E_sar: (60, 72)
Shape k: (60,)
Shape phi: (72,)
╔══════════════════════════════════════════════════════════════╗
║         CONVERSÃO SAR: m⁴ → m²·s·rad⁻¹ (WW3 units)          ║
╠══════════════════════════════════════════════════════════════╣
║ Shape:                                             (60, 72) ║
║ Frequências: 60 bins | Direções: 72 bins              ║
║ Freq range: 0.0361 - 0.2281 Hz                       ║
║ Dir range: 0.0° - 355.0°                            ║
╟──────────────────────────────────────────────────────────────╢
║ Jacobiano dk/df: 0.2903 - 1.8361                   ║
║ Fator angular (π/180): 0.017453                      ║
╟

/Users/jtakeo/googleDrive/myProjects/sar-spec-partitioning/scripts/utils.py:139: UserWarning: no explicit representation of timezones available for np.datetime64
  else:


In [45]:
# DIAGNÓSTICO: Comparação com WW3
print("="*70)
print("COMPARAÇÃO SAR vs WW3")
print("="*70)
print(f"WW3  -> Hs: 2.74 m, Tp: 12.8 s, Dp: 195°")
print(f"SAR  -> Hs: {hs:.2f} m, Tp: {tp:.2f} s, Dp: {dp:.2f}°")
print("-"*70)

# Calcular fator de correção baseado em Hs
hs_ww3 = 2.74
m0_ww3 = (hs_ww3 / 4)**2
m0_sar_current = (hs / 4)**2

fator_correcao = m0_ww3 / m0_sar_current
print(f"\nAnálise de momentos espectrais:")
print(f"  m0_WW3 = {m0_ww3:.6f} m²")
print(f"  m0_SAR = {m0:.6f} m²")
print(f"  m0_SAR (via Hs) = {m0_sar_current:.6f} m²")
print(f"  Fator de correção necessário: {fator_correcao:.6f}")
print(f"  Sqrt do fator: {np.sqrt(fator_correcao):.6f}")

# Checar integração
print("\n" + "-"*70)
print("Verificação da integração numérica:")
ddir_deg = 360.0 / len(dirs)
print(f"  ddir = {ddir:.6f} rad ({ddir_deg:.2f}°)")
print(f"  df médio = {np.mean(delf):.6f} Hz")
print(f"  df min/max = {np.min(delf):.6f} / {np.max(delf):.6f} Hz")

# Soma total de energia
energia_total = np.sum(E2d) * ddir * np.mean(delf)
print(f"\n  Energia total integrada: {energia_total:.6f} m²")
print(f"  m0 calculado: {m0:.6f} m²")
print(f"  Diferença: {abs(energia_total - m0):.6e} m²")

# Verificar unidades
print("\n" + "-"*70)
print("Verificação dimensional:")
print(f"  E2d tem unidades: m²·s·rad⁻¹")
print(f"  E2d × ddir[rad] × df[Hz] = m²·s·rad⁻¹ × rad × s⁻¹ = m² ✓")
print("="*70)

COMPARAÇÃO SAR vs WW3
WW3  -> Hs: 2.74 m, Tp: 12.8 s, Dp: 195°
SAR  -> Hs: 1.25 m, Tp: 13.51 s, Dp: 180.00°
----------------------------------------------------------------------

Análise de momentos espectrais:
  m0_WW3 = 0.469225 m²
  m0_SAR = 0.097210 m²
  m0_SAR (via Hs) = 0.097210 m²
  Fator de correção necessário: 4.826924
  Sqrt do fator: 2.197026

----------------------------------------------------------------------
Verificação da integração numérica:
  ddir = 0.087266 rad (5.00°)
  df médio = 0.003318 Hz
  df min/max = 0.001145 / 0.007021 Hz

  Energia total integrada: 0.152964 m²
  m0 calculado: 0.097210 m²
  Diferença: 5.575358e-02 m²

----------------------------------------------------------------------
Verificação dimensional:
  E2d tem unidades: m²·s·rad⁻¹
  E2d × ddir[rad] × df[Hz] = m²·s·rad⁻¹ × rad × s⁻¹ = m² ✓


In [47]:
# Testar diferentes métodos de integração
print("TESTE DE MÉTODOS DE INTEGRAÇÃO")
print("="*70)

# Método 1: Integração direta 2D
E2d_clean = np.where(np.isfinite(E2d) & (E2d >= 0), E2d, 0)
m0_method1 = np.sum(E2d_clean * ddir) * np.mean(delf)
hs_method1 = 4 * np.sqrt(m0_method1)
print(f"\nMétodo 1 (integração direta 2D):")
print(f"  m0 = {m0_method1:.6f} m²")
print(f"  Hs = {hs_method1:.6f} m")

# Método 2: Via espectro 1D (atual)
spec1d = np.sum(E2d_clean, axis=1) * ddir
m0_method2 = np.sum(spec1d * delf)
hs_method2 = 4 * np.sqrt(m0_method2)
print(f"\nMétodo 2 (via espectro 1D - atual):")
print(f"  m0 = {m0_method2:.6f} m²")
print(f"  Hs = {hs_method2:.6f} m")

# Método 3: Trapézio 2D usando numpy
m0_method3 = 0
for j in range(len(dirs)):
    m0_method3 += np.trapz(E2d_clean[:, j], freq) * ddir
hs_method3 = 4 * np.sqrt(m0_method3)
print(f"\nMétodo 3 (trapézio em freq, soma em dir):")
print(f"  m0 = {m0_method3:.6f} m²")
print(f"  Hs = {hs_method3:.6f} m")

# Método 4: Usando incrementos corretos para cada frequência
m0_method4 = 0
for i in range(len(freq)):
    for j in range(len(dirs)):
        m0_method4 += E2d_clean[i, j] * delf[i] * ddir
hs_method4 = 4 * np.sqrt(m0_method4)
print(f"\nMétodo 4 (incrementos individuais):")
print(f"  m0 = {m0_method4:.6f} m²")
print(f"  Hs = {hs_method4:.6f} m")

print("\n" + "="*70)
print(f"Referência WW3: Hs = 2.74 m (m0 = {(2.74/4)**2:.6f} m²)")
print("="*70)

TESTE DE MÉTODOS DE INTEGRAÇÃO

Método 1 (integração direta 2D):
  m0 = 0.152964 m²
  Hs = 1.564422 m

Método 2 (via espectro 1D - atual):
  m0 = 0.097210 m²
  Hs = 1.247140 m

Método 3 (trapézio em freq, soma em dir):
  m0 = 0.095705 m²
  Hs = 1.237450 m

Método 4 (incrementos individuais):
  m0 = 0.097210 m²
  Hs = 1.247140 m

Referência WW3: Hs = 2.74 m (m0 = 0.469225 m²)


/var/folders/cp/8th3fpl14tz3yqc3hyy8bksc0000gq/T/ipykernel_33217/4192995184.py:24: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  m0_method3 += np.trapz(E2d_clean[:, j], freq) * ddir


In [48]:
# Investigar fatores de normalização conhecidos
print("\nINVESTIGAÇÃO DE FATORES DE NORMALIZAÇÃO")
print("="*70)

# Fator necessário para igualar WW3
fator_necessario = 2.74 / 1.564422  # WW3 / SAR melhor método
print(f"Fator multiplicativo necessário: {fator_necessario:.6f}")
print(f"Fator ao quadrado (para energia): {fator_necessario**2:.6f}")

# Testar fatores conhecidos
fatores_teoricos = {
    "2π (freq angular ↔ Hz)": 2 * np.pi,
    "√2 (rms ↔ amplitude)": np.sqrt(2),
    "2 (energia bilateral)": 2.0,
    "4π (área esfera unitária)": 4 * np.pi,
    "π (semicírculo)": np.pi,
    "180/π (rad↔deg energia)": 180.0 / np.pi,
    "Combinação 2π×√2": 2 * np.pi * np.sqrt(2),
}

print("\nFatores teóricos candidatos:")
for nome, fator in fatores_teoricos.items():
    hs_test = 1.564422 * fator
    erro = abs(hs_test - 2.74) / 2.74 * 100
    print(f"  {nome:30s}: {fator:.6f} → Hs = {hs_test:.3f} m (erro: {erro:.1f}%)")

print("\n" + "="*70)

# Verificar se há alguma variável de calibração no arquivo SAR
print("\nVariáveis do arquivo SAR:")
for var in ds.variables:
    print(f"  {var}")
    
# Verificar atributos da variável wave_spec
if 'wave_spec' in ds.variables:
    print("\nAtributos de 'wave_spec':")
    for attr in ds['wave_spec'].attrs:
        print(f"  {attr}: {ds['wave_spec'].attrs[attr]}")


INVESTIGAÇÃO DE FATORES DE NORMALIZAÇÃO
Fator multiplicativo necessário: 1.751446
Fator ao quadrado (para energia): 3.067562

Fatores teóricos candidatos:
  2π (freq angular ↔ Hz)        : 6.283185 → Hs = 9.830 m (erro: 258.7%)
  √2 (rms ↔ amplitude)          : 1.414214 → Hs = 2.212 m (erro: 19.3%)
  2 (energia bilateral)         : 2.000000 → Hs = 3.129 m (erro: 14.2%)
  4π (área esfera unitária)     : 12.566371 → Hs = 19.659 m (erro: 617.5%)
  π (semicírculo)               : 3.141593 → Hs = 4.915 m (erro: 79.4%)
  180/π (rad↔deg energia)       : 57.295780 → Hs = 89.635 m (erro: 3171.3%)
  Combinação 2π×√2              : 8.885766 → Hs = 13.901 m (erro: 407.3%)


Variáveis do arquivo SAR:
  VAVH
  VPED
  VTPK
  time
  longitude
  latitude
  wave_spec
  partition_domain_spec
  wavenumber_spec
  direction_spec
  nv
  inten
  snr
  resolution_spec
  VPED_flag
  VTPK_flag
  VAVH_flag
  L2_source_product
  L2_partition_quality_flag

Atributos de 'wave_spec':
  units: m^4
  long_name: Estima

In [49]:
# Comparar com Hs direto do arquivo SAR
print("\nCOMPARAÇÃO COM PARÂMETROS DO ARQUIVO SAR")
print("="*70)

vavh_sar = ds.VAVH[index].values
vtpk_sar = ds.VTPK[index].values
vped_sar = ds.VPED[index].values

print(f"Parâmetros integrados do arquivo SAR (index={index}):")
print(f"  VAVH (Hs): {vavh_sar:.2f} m")
print(f"  VTPK (Tp): {vtpk_sar:.2f} s")
print(f"  VPED (Dp): {vped_sar:.2f}°")

print(f"\nNosso cálculo (conversão atual):")
print(f"  Hs: {hs:.2f} m")
print(f"  Tp: {tp:.2f} s")
print(f"  Dp: {dp:.2f}°")

print(f"\nWW3 (referência):")
print(f"  Hs: 2.74 m")
print(f"  Tp: 12.8 s")
print(f"  Dp: 195°")

# Calcular fator baseado no Hs do arquivo
if vavh_sar > 0 and hs > 0:
    fator_sar_file = vavh_sar / hs
    print(f"\n{'─'*70}")
    print(f"Fator de correção (VAVH_SAR / Hs_calculado): {fator_sar_file:.6f}")
    print(f"Fator ao quadrado (para energia): {fator_sar_file**2:.6f}")
    
    # Testar aplicação do fator
    hs_corrigido = hs * fator_sar_file
    m0_corrigido = m0 * (fator_sar_file**2)
    print(f"\nAplicando fator:")
    print(f"  Hs corrigido: {hs_corrigido:.2f} m")
    print(f"  m0 corrigido: {m0_corrigido:.6f} m²")
    print(f"  Erro vs WW3: {abs(hs_corrigido - 2.74)/2.74*100:.1f}%")

print("="*70)


COMPARAÇÃO COM PARÂMETROS DO ARQUIVO SAR
Parâmetros integrados do arquivo SAR (index=223):
  VAVH (Hs): 3.08 m
  VTPK (Tp): 15.64 s
  VPED (Dp): 31.92°

Nosso cálculo (conversão atual):
  Hs: 1.25 m
  Tp: 13.51 s
  Dp: 180.00°

WW3 (referência):
  Hs: 2.74 m
  Tp: 12.8 s
  Dp: 195°

──────────────────────────────────────────────────────────────────────
Fator de correção (VAVH_SAR / Hs_calculado): 2.473576
Fator ao quadrado (para energia): 6.118577

Aplicando fator:
  Hs corrigido: 3.08 m
  m0 corrigido: 0.594787 m²
  Erro vs WW3: 12.6%


In [50]:
# ANÁLISE TEÓRICA DETALHADA DA CONVERSÃO
print("\n" + "="*80)
print(" ANÁLISE DETALHADA: E(k,θ) [m⁴] → E(f,θ) [m²·s·rad⁻¹]")
print("="*80)

# Recarregar dados brutos do SAR
E_sar_raw = ds.wave_spec.values[:, :, index]
k_raw = ds.wavenumber_spec.values
phi_raw = ds.direction_spec.values

print(f"\nDados brutos SAR:")
print(f"  Shape E_sar: {E_sar_raw.shape}")
print(f"  Shape k: {k_raw.shape}")
print(f"  Shape φ: {phi_raw.shape}")
print(f"  Integral E_sar total: {np.sum(E_sar_raw):.6e} m⁴")

# Conversão passo a passo
g = 9.81
omega = np.sqrt(g * k_raw)
freq_calc = omega / (2 * np.pi)

# Jacobiano
dkdf = 8 * np.pi**2 * freq_calc / g
print(f"\n1. Jacobiano dk/df:")
print(f"   Range: {np.min(dkdf):.6f} - {np.max(dkdf):.6f}")
print(f"   Fórmula: dk/df = 8π²f/g")

# Conversão angular - CRÍTICO!
print(f"\n2. Conversão angular:")
print(f"   φ em graus → radianos")
print(f"   Δφ_graus = {360/len(phi_raw):.2f}°")
print(f"   Δφ_rad = {2*np.pi/len(phi_raw):.6f} rad")
print(f"   Relação: Δφ_rad = Δφ_deg × (π/180)")

# TESTE: E(k,θ) com θ em GRAUS vs RADIANOS
print(f"\n3. Interpretação das unidades:")
print(f"   OPÇÃO A: E_SAR está em [m⁴/(rad·m⁻¹)]")
print(f"     → Multiplicar por π/180 para converter θ")
print(f"   OPÇÃO B: E_SAR está em [m⁴/(deg·m⁻¹)]")
print(f"     → NÃO multiplicar, apenas converter eixo")

# Testar ambas opções
E_opcao_A = E_sar_raw * dkdf.reshape(-1, 1) * (np.pi/180)
E_opcao_B = E_sar_raw * dkdf.reshape(-1, 1)

# Integrar ambas
ddir_rad = 2*np.pi / len(phi_raw)
df = np.zeros_like(freq_calc)
for i in range(len(freq_calc)-1):
    df[i] = freq_calc[i+1] - freq_calc[i]
df[-1] = df[-2]

m0_A = np.sum(E_opcao_A) * ddir_rad * np.mean(df)
hs_A = 4 * np.sqrt(m0_A)

m0_B = np.sum(E_opcao_B) * ddir_rad * np.mean(df)
hs_B = 4 * np.sqrt(m0_B)

print(f"\nRESULTADOS:")
print(f"{'─'*80}")
print(f"  OPÇÃO A (com π/180):  m0 = {m0_A:.6f} m² → Hs = {hs_A:.2f} m")
print(f"  OPÇÃO B (sem π/180):  m0 = {m0_B:.6f} m² → Hs = {hs_B:.2f} m")
print(f"{'─'*80}")
print(f"  VAVH do arquivo SAR: {vavh_sar:.2f} m")
print(f"  WW3:                 2.74 m")
print(f"{'─'*80}")
print(f"  Erro A vs VAVH: {abs(hs_A - vavh_sar)/vavh_sar*100:.1f}%")
print(f"  Erro B vs VAVH: {abs(hs_B - vavh_sar)/vavh_sar*100:.1f}%")
print("="*80)


 ANÁLISE DETALHADA: E(k,θ) [m⁴] → E(f,θ) [m²·s·rad⁻¹]

Dados brutos SAR:
  Shape E_sar: (60, 72)
  Shape k: (60,)
  Shape φ: (72,)
  Integral E_sar total: 3.270003e+05 m⁴

1. Jacobiano dk/df:
   Range: 0.290318 - 1.836135
   Fórmula: dk/df = 8π²f/g

2. Conversão angular:
   φ em graus → radianos
   Δφ_graus = 5.00°
   Δφ_rad = 0.087266 rad
   Relação: Δφ_rad = Δφ_deg × (π/180)

3. Interpretação das unidades:
   OPÇÃO A: E_SAR está em [m⁴/(rad·m⁻¹)]
     → Multiplicar por π/180 para converter θ
   OPÇÃO B: E_SAR está em [m⁴/(deg·m⁻¹)]
     → NÃO multiplicar, apenas converter eixo

RESULTADOS:
────────────────────────────────────────────────────────────────────────────────
  OPÇÃO A (com π/180):  m0 = 0.883518 m² → Hs = 3.76 m
  OPÇÃO B (sem π/180):  m0 = 50.621841 m² → Hs = 28.46 m
────────────────────────────────────────────────────────────────────────────────
  VAVH do arquivo SAR: 3.08 m
  WW3:                 2.74 m
──────────────────────────────────────────────────────────────────

In [51]:
# CORREÇÃO DO MÉTODO DE INTEGRAÇÃO
print("\n" + "🔍 TESTE DE DIFERENTES MÉTODOS DE INTEGRAÇÃO")
print("="*80)

# Método correto: soma com incrementos individuais
m0_correto_A = 0
for i in range(len(freq_calc)):
    for j in range(len(phi_raw)):
        m0_correto_A += E_opcao_A[i, j] * df[i] * ddir_rad

hs_correto_A = 4 * np.sqrt(m0_correto_A)

print(f"OPÇÃO A - Integração correta (soma df[i]):")
print(f"  m0 = {m0_correto_A:.6f} m²")
print(f"  Hs = {hs_correto_A:.2f} m")
print(f"  Erro vs VAVH (3.08m): {abs(hs_correto_A - 3.08)/3.08*100:.1f}%")
print(f"  Erro vs WW3 (2.74m):  {abs(hs_correto_A - 2.74)/2.74*100:.1f}%")

# Método trapézio
m0_trapz_A = 0
for j in range(len(phi_raw)):
    m0_trapz_A += np.trapezoid(E_opcao_A[:, j], freq_calc) * ddir_rad

hs_trapz_A = 4 * np.sqrt(m0_trapz_A)

print(f"\nOPÇÃO A - Integração trapézio:")
print(f"  m0 = {m0_trapz_A:.6f} m²")
print(f"  Hs = {hs_trapz_A:.2f} m")
print(f"  Erro vs VAVH (3.08m): {abs(hs_trapz_A - 3.08)/3.08*100:.1f}%")
print(f"  Erro vs WW3 (2.74m):  {abs(hs_trapz_A - 2.74)/2.74*100:.1f}%")

print("\n" + "="*80)
print("✅ CONCLUSÃO: A conversão correta usa (π/180) e integração com trapézio!")
print("="*80)


🔍 TESTE DE DIFERENTES MÉTODOS DE INTEGRAÇÃO
OPÇÃO A - Integração correta (soma df[i]):
  m0 = 0.569915 m²
  Hs = 3.02 m
  Erro vs VAVH (3.08m): 2.0%
  Erro vs WW3 (2.74m):  10.2%

OPÇÃO A - Integração trapézio:
  m0 = 0.561141 m²
  Hs = 3.00 m
  Erro vs VAVH (3.08m): 2.7%
  Erro vs WW3 (2.74m):  9.4%

✅ CONCLUSÃO: A conversão correta usa (π/180) e integração com trapézio!


In [59]:
# Comparar E2d retornado pela função vs cálculo manual
print("\nCOMPARAÇÃO: E2d da função vs cálculo manual")
print("="*80)
print(f"E2d (da função load_sar_spectrum):")
print(f"  Shape: {E2d.shape}")
print(f"  Range: {np.min(E2d):.6e} - {np.max(E2d):.6e}")
print(f"  Soma total: {np.sum(E2d):.6e}")

print(f"\nE_opcao_A (cálculo manual correto):")
print(f"  Shape: {E_opcao_A.shape}")
print(f"  Range: {np.min(E_opcao_A):.6e} - {np.max(E_opcao_A):.6e}")
print(f"  Soma total: {np.sum(E_opcao_A):.6e}")

print(f"\nRazão E_opcao_A / E2d: {np.sum(E_opcao_A) / np.sum(E2d):.6f}")
print("="*80)


COMPARAÇÃO: E2d da função vs cálculo manual
E2d (da função load_sar_spectrum):
  Shape: (60, 72)
  Range: 0.000000e+00 - 3.705964e+00
  Soma total: 5.282756e+02

E_opcao_A (cálculo manual correto):
  Shape: (60, 72)
  Range: 0.000000e+00 - 4.808760e+01
  Soma total: 3.051322e+03

Razão E_opcao_A / E2d: 5.776003


In [56]:
# ANÁLISE DEFINITIVA: Qual é o fator correto?
print("\n" + "="*80)
print(" DETERMINAÇÃO DO FATOR CORRETO")
print("="*80)

# Dados originais
print(f"\nE_sar [m⁴] original: sum = {np.sum(E_sar_raw):.6e}")

# Teste 1: Apenas jacobiano (sem conversão angular)
E_test1 = E_sar_raw * dkdf.reshape(-1, 1)
m0_test1 = 0
for j in range(len(phi_raw)):
    m0_test1 += np.trapezoid(E_test1[:, j], freq_calc) * ddir_rad
hs_test1 = 4 * np.sqrt(m0_test1)
print(f"\nTESTE 1 - Apenas jacobiano dk/df:")
print(f"  E × dk/df → m0 = {m0_test1:.6f} m², Hs = {hs_test1:.2f} m")

# Teste 2: Jacobiano × π/180
E_test2 = E_sar_raw * dkdf.reshape(-1, 1) * (np.pi/180)
m0_test2 = 0
for j in range(len(phi_raw)):
    m0_test2 += np.trapezoid(E_test2[:, j], freq_calc) * ddir_rad
hs_test2 = 4 * np.sqrt(m0_test2)
print(f"\nTESTE 2 - Jacobiano × π/180:")
print(f"  E × dk/df × π/180 → m0 = {m0_test2:.6f} m², Hs = {hs_test2:.2f} m")

# Teste 3: Jacobiano × 180/π
E_test3 = E_sar_raw * dkdf.reshape(-1, 1) * (180/np.pi)
m0_test3 = 0
for j in range(len(phi_raw)):
    m0_test3 += np.trapezoid(E_test3[:, j], freq_calc) * ddir_rad
hs_test3 = 4 * np.sqrt(m0_test3)
print(f"\nTESTE 3 - Jacobiano × 180/π:")
print(f"  E × dk/df × 180/π → m0 = {m0_test3:.6f} m², Hs = {hs_test3:.2f} m")

print("\n" + "="*80)
print(f"Referências:")
print(f"  VAVH (SAR file): {vavh_sar:.2f} m")
print(f"  WW3:             2.74 m")
print("="*80)
print(f"Melhor match: TESTE 2 (π/180) com erro de {abs(hs_test2-vavh_sar)/vavh_sar*100:.1f}% vs VAVH")
print("="*80)


 DETERMINAÇÃO DO FATOR CORRETO

E_sar [m⁴] original: sum = 3.270003e+05

TESTE 1 - Apenas jacobiano dk/df:
  E × dk/df → m0 = 32.151001 m², Hs = 22.68 m

TESTE 2 - Jacobiano × π/180:
  E × dk/df × π/180 → m0 = 0.561141 m², Hs = 3.00 m

TESTE 3 - Jacobiano × 180/π:
  E × dk/df × 180/π → m0 = 1842.116577 m², Hs = 171.68 m

Referências:
  VAVH (SAR file): 3.08 m
  WW3:             2.74 m
Melhor match: TESTE 2 (π/180) com erro de 2.9% vs VAVH


In [60]:
# DEBUG: Verificar se os dados são os mesmos
print("\nVERIFICAÇÃO DE DADOS:")
print("="*80)
print(f"E_sar_raw (do notebook manual):")
print(f"  Shape: {E_sar_raw.shape}")
print(f"  Sum: {np.sum(E_sar_raw):.6e}")
print(f"  Max: {np.max(E_sar_raw):.6e}")

print(f"\nRecarregar E_sar via função:")
# Recarregar direto do arquivo para comparar
E_sar_function = ds.wave_spec.values[:, :, index]
print(f"  Shape: {E_sar_function.shape}")
print(f"  Sum: {np.sum(E_sar_function):.6e}")
print(f"  Max: {np.max(E_sar_function):.6e}")

print(f"\nSão idênticos? {np.allclose(E_sar_raw, E_sar_function)}")

# Aplicar manualmente a conversão completa
E_manual_full = E_sar_function * dkdf.reshape(-1, 1) * (np.pi/180)
print(f"\nE_manual_full (aplicando conversão manualmente):")
print(f"  Shape: {E_manual_full.shape}")
print(f"  Sum: {np.sum(E_manual_full):.6e}")
print(f"  Max: {np.max(E_manual_full):.6e}")

print(f"\nE2d (retornado pela função):")
print(f"  Shape: {E2d.shape}")
print(f"  Sum: {np.sum(E2d):.6e}")
print(f"  Max: {np.max(E2d):.6e}")

print(f"\nRazão E_manual_full / E2d: {np.sum(E_manual_full) / np.sum(E2d):.6f}")
print("="*80)


VERIFICAÇÃO DE DADOS:
E_sar_raw (do notebook manual):
  Shape: (60, 72)
  Sum: 3.270003e+05
  Max: 5.344018e+03

Recarregar E_sar via função:
  Shape: (60, 72)
  Sum: 3.270003e+05
  Max: 5.344018e+03

São idênticos? True

E_manual_full (aplicando conversão manualmente):
  Shape: (60, 72)
  Sum: 3.051322e+03
  Max: 4.808760e+01

E2d (retornado pela função):
  Shape: (60, 72)
  Sum: 5.282756e+02
  Max: 3.705964e+00

Razão E_manual_full / E2d: 5.776003


In [62]:
# DEBUG: Verificar shape do wave_spec
print("\nANÁLISE DO SHAPE wave_spec:")
print("="*80)
print(f"ds.wave_spec.shape: {ds.wave_spec.shape}")
print(f"ds.wavenumber_spec.shape: {ds.wavenumber_spec.shape}")
print(f"ds.direction_spec.shape: {ds.direction_spec.shape}")
print(f"ds.time.shape: {ds.time.shape}")

print(f"\nE_sar_raw = ds.wave_spec.values[:, :, {index}]")
print(f"  Shape: {E_sar_raw.shape}")
print(f"  Sum: {np.sum(E_sar_raw):.6e}")

# Teste de diferentes slices
E_test_slice1 = ds.wave_spec.values[:, :, index]
E_test_slice2 = np.squeeze(ds.wave_spec.values[:, :, index])

print(f"\nSlice [:, :, {index}]:")
print(f"  Shape: {E_test_slice1.shape}")
print(f"  Sum: {np.sum(E_test_slice1):.6e}")

print(f"\nnp.squeeze(slice [:, :, {index}]):")
print(f"  Shape: {E_test_slice2.shape}")
print(f"  Sum: {np.sum(E_test_slice2):.6e}")

# Verificar se é problema de transpose
E_transposed = E_sar_raw.T
print(f"\nE_sar_raw.T:")
print(f"  Shape: {E_transposed.shape}")
print(f"  Sum: {np.sum(E_transposed):.6e}")
print("="*80)


ANÁLISE DO SHAPE wave_spec:
ds.wave_spec.shape: (60, 72, 2389)
ds.wavenumber_spec.shape: (60,)
ds.direction_spec.shape: (72,)
ds.time.shape: (2389,)

E_sar_raw = ds.wave_spec.values[:, :, 223]
  Shape: (60, 72)
  Sum: 3.270003e+05

Slice [:, :, 223]:
  Shape: (60, 72)
  Sum: 3.270003e+05

np.squeeze(slice [:, :, 223]):
  Shape: (60, 72)
  Sum: 3.270003e+05

E_sar_raw.T:
  Shape: (72, 60)
  Sum: 3.270003e+05


In [65]:
# DEBUG: Verificar o problema de índice
print("\nVERIFICAÇÃO DE ÍNDICE:")
print("="*80)
print(f"index passado: {index}")
print(f"date_time passado: {date_time}")

# Buscar índice mais próximo do date_time
import pandas as pd
dt = pd.to_datetime(date_time)
times = ds.time.values
idx_closest = abs(times - np.datetime64(dt)).argmin()

print(f"\nBusca por date_time '{date_time}':")
print(f"  Índice encontrado: {idx_closest}")
print(f"  Horário encontrado: {pd.to_datetime(times[idx_closest])}")

print(f"\nUsando index={index}:")
print(f"  Horário: {pd.to_datetime(times[index])}")

print(f"\nDiferença: {idx_closest} vs {index} = {idx_closest - index} passos")
print(f"\nE_sar sums:")
print(f"  E_sar[:, :, {idx_closest}] sum: {np.sum(ds.wave_spec.values[:, :, idx_closest]):.6e}")
print(f"  E_sar[:, :, {index}] sum: {np.sum(ds.wave_spec.values[:, :, index]):.6e}")
print("="*80)


VERIFICAÇÃO DE ÍNDICE:
index passado: 223
date_time passado: 2021-04-28T20:00:00Z

Busca por date_time '2021-04-28T20:00:00Z':
  Índice encontrado: 234
  Horário encontrado: 2021-04-28 20:00:09

Usando index=223:
  Horário: 2021-04-28 19:56:44

Diferença: 234 vs 223 = 11 passos

E_sar sums:
  E_sar[:, :, 234] sum: 5.972998e+04
  E_sar[:, :, 223] sum: 3.270003e+05


/var/folders/cp/8th3fpl14tz3yqc3hyy8bksc0000gq/T/ipykernel_33217/731147868.py:11: UserWarning: no explicit representation of timezones available for np.datetime64
  idx_closest = abs(times - np.datetime64(dt)).argmin()


In [66]:
# TESTE FINAL: Aplicar conversão no índice correto (234)
print("\nTESTE COM ÍNDICE CORRETO (234) - 2021-04-28 20:00:09")
print("="*80)

E_sar_idx234 = ds.wave_spec.values[:, :, 234]
k_use = ds.wavenumber_spec.values
phi_use = ds.direction_spec.values

# Conversão correta
g = 9.81
omega = np.sqrt(g * k_use)
freq_use = omega / (2 * np.pi)
dkdf_use = 8 * np.pi**2 * freq_use / g

E_converted_234 = E_sar_idx234 * dkdf_use.reshape(-1, 1) * (np.pi/180)

# Integrar com trapézio
ddir_use = 2*np.pi / len(phi_use)
m0_234 = 0
for j in range(len(phi_use)):
    m0_234 += np.trapezoid(E_converted_234[:, j], freq_use) * ddir_use

hs_234 = 4 * np.sqrt(m0_234)

# Comparar com VAVH do arquivo
vavh_234 = ds.VAVH[234].values
vtpk_234 = ds.VTPK[234].values
vped_234 = ds.VPED[234].values

print(f"Calculado (conversão correta):")
print(f"  m0 = {m0_234:.6f} m²")
print(f"  Hs = {hs_234:.2f} m")

print(f"\nVAVH do arquivo SAR (idx=234):")
print(f"  Hs = {vavh_234:.2f} m")
print(f"  Tp = {vtpk_234:.2f} s")
print(f"  Dp = {vped_234:.2f}°")

print(f"\nWW3 (2021-04-28 20:00:00):")
print(f"  Hs = 2.74 m")
print(f"  Tp = 12.8 s")
print(f"  Dp = 195°")

print(f"\nERROS:")
print(f"  Hs calculado vs VAVH: {abs(hs_234 - vavh_234)/vavh_234*100:.1f}%")
print(f"  Hs calculado vs WW3:   {abs(hs_234 - 2.74)/2.74*100:.1f}%")
print(f"  VAVH vs WW3:           {abs(vavh_234 - 2.74)/2.74*100:.1f}%")
print("="*80)


TESTE COM ÍNDICE CORRETO (234) - 2021-04-28 20:00:09
Calculado (conversão correta):
  m0 = 0.095705 m²
  Hs = 1.24 m

VAVH do arquivo SAR (idx=234):
  Hs = 1.12 m
  Tp = 13.85 s
  Dp = 351.77°

WW3 (2021-04-28 20:00:00):
  Hs = 2.74 m
  Tp = 12.8 s
  Dp = 195°

ERROS:
  Hs calculado vs VAVH: 10.7%
  Hs calculado vs WW3:   54.8%
  VAVH vs WW3:           59.2%
